# NID (NSO in Docker) demo
## 🍜 NID base and dev images 🍜
---
### Just add water, stir and enjoy

The NSO in Docker (NID) project aims to provide a container-state-of-mind approach to the way in which we maintain staging, development, testing and (why not) production environments.

The holistic container-first approach makes it easy to isolate the purpose of every component, so that each container is dedicated to a specific function only.

The following steps describe the image creation, container spinning, and the different features of each flavor.

First of all, clone the official repository in your host:

```
git clone https://github.com/NSO-developer/nso-docker
```

Download the Cisco NSO free Linux signed.bin image for testing purposes from [this link](https://software.cisco.com/download/home/286331591/type/286283941/release/6.0). The version currently available is v6.0. Once downloaded, issue the following command to extract the installer file:

```
% sh nso-6.0.linux.x86_64.signed.bin . . .
nso-6.0.linux.x86_64.installer.bin
```

This will generate a series of files. Locate the one which ends in .installer.bin and place it in the nso-install-files/ directory of the Docker for NSO repository.

Once done, issue the following command in the root location of the directory:

```
% make
```

This will compile two different flavors of NSO docker images into your local collection. You can verify the completion with the following command:

In [1]:
!docker images | grep cisco-nso

cisco-nso-base                                                   6.0-root               a173e5c9baa9   3 months ago    678MB
cisco-nso-dev                                                    6.0-root               825b256d44ac   3 months ago    1.43GB


### 🍜 ```base``` image 🍜

The base image has the following properties:
- Automatic booting of the NSO server within (of course it takes some time to boot)
- No possibility to run NETSIM witin
- Ideal for usage in production environments, along with persistent mounted volumes for CDB, rollbacks, backups, etc
- It is possible to connect other resources such as external netsim devices or staging topologies using Docker networks

### 🍜 ```dev``` image 🍜

The dev image has the following properties:
- Workbech purposes
- NSO must be booted up manually
- Possbility to run NETSIM and host devices within the same container

### 💬 An idiomatic approach to spinning up containers 💬

The image that we just generated with spin up two different kinds of NSO containers. However, there are different ways in which we can specify which are the features that the NSO server within will have.

In this demo, we will explore the idiomatic parameters which can change the configs within ```/etc/ncs/ncs.conf```. These can be specified in a separate file and passed as an argument when spinning the container.

What will happen is that the ncs.conf file will have its settings mangled prior to booting.

You can check an example [in this file within the repository](/pipeline-utils/nso_setup.list).

The file must be passed along with the ```--env-file``` flag when spinning the container.

### 🔌 Exposed ports by default 🔌

<table border="2" cellspacing="0" cellpadding="6" rules="groups" frame="hsides">


<colgroup>
<col  class="org-left" />

<col  class="org-right" />

<col  class="org-left" />

<col  class="org-left" />
</colgroup>
<thead>
<tr>
<th scope="col" class="org-left">Protocol</th>
<th scope="col" class="org-right">Port</th>
<th scope="col" class="org-left">Use</th>
<th scope="col" class="org-left">Config var</th>
</tr>
</thead>

<tbody>
<tr>
<td class="org-left">TCP</td>
<td class="org-right">22</td>
<td class="org-left">SSH</td>
<td class="org-left"><code>SSH_PORT</code></td>
</tr>


<tr>
<td class="org-left">TCP</td>
<td class="org-right">80</td>
<td class="org-left">HTTP</td>
<td class="org-left">&#xa0;</td>
</tr>


<tr>
<td class="org-left">TCP</td>
<td class="org-right">443</td>
<td class="org-left">HTTPS</td>
<td class="org-left">&#xa0;</td>
</tr>


<tr>
<td class="org-left">TCP</td>
<td class="org-right">830</td>
<td class="org-left">NETCONF</td>
<td class="org-left">&#xa0;</td>
</tr>


<tr>
<td class="org-left">TCP</td>
<td class="org-right">4334</td>
<td class="org-left">NETCONF call-home</td>
<td class="org-left">&#xa0;</td>
</tr>


<tr>
<td class="org-left">TCP</td>
<td class="org-right">4570</td>
<td class="org-left">NSO HA</td>
<td class="org-left">&#xa0;</td>
</tr>
</tbody>
</table>

### 🍽️ Cooking time 🍽️

Let's spin a container using the ```base``` image. 

For that purpose, we will use the idiomatic approach file contained in this repository. We will open the HTTP ports for accesing the RESTCONF interface:

In [8]:
!!docker run -itd --env-file pipeline-utils/nso_setup.list --platform=linux/amd64 -p 8081:443 --name MYNSO-BASE cisco-nso-base:6.0-root

['596a8a0a4ac21213ba987630a195f524a0996d5fa799761bc684d866b8de75bd']

After waiting for some time for NSO to boot, let's copy, compile and onboard the sample service contained in this repository.

**VERY IMPORTANT!** The default location of our packages within this container is ```/nso/run/packages/```. This can be changed so that another location or even a mounted volume are default.

In [9]:
!docker cp rfs-common MYNSO-BASE:/nso/run/packages

In [10]:
!docker exec -i MYNSO-BASE bash -l -c "cd /nso/run/packages/rfs-common/src/ && make clean all"

rm -rf ../load-dir java/src//
mkdir -p ../load-dir
mkdir -p java/src//
/opt/ncs/ncs-6.0/bin/ncsc  `ls rfs-common-ann.yang  > /dev/null 2>&1 && echo "-a rfs-common-ann.yang"` \
	--fail-on-warnings \
	 \
	-c -o ../load-dir/rfs-common.fxs yang/rfs-common.yang


Finally, time to issue a packages reload!

In [11]:
!docker exec -i MYNSO-BASE bash -l -c "echo 'packages reload' | ncs_cli -Cu admin"


>>> System upgrade is starting.
>>> Sessions in configure mode must exit to operational mode.
>>> No configuration changes can be performed until upgrade has completed.
>>> System upgrade has completed successfully.
reload-result {
    package rfs-common
    result true
}


The container is ready to be used!

In [12]:
!docker exec -i MYNSO-BASE bash -l -c "echo 'show packages package * package-version' | ncs_cli -Cu admin"

            PACKAGE  
NAME        VERSION  
---------------------
rfs-common  1.0      

